In [24]:
import math
from dataclasses import dataclass
from pathlib import Path
from collections import deque, defaultdict
from matplotlib.ticker import MultipleLocator, FuncFormatter

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO


# ------------------------------
# НАСТРОЙКИ
# ------------------------------

# Путь к видеофайлу
VIDEO_PATH = "2025-12-16 14-42-07.mov"

# Список моделей, которые будем сравнивать
MODEL_WEIGHTS_LIST = [
    "yolo11n.pt",
    "yolo11s.pt",
    "yolo11m.pt",
    "yolo11l.pt",
    "yolo11x.pt",
]

# Порог уверенности (меньше — больше детекций, но больше мусора)
CONF_THRES = 0.25
# Порог IoU для подавления/трекера (влияет на стабильность)
IOU_THRES = 0.6
# Обрабатываем не каждый кадр, а с шагом (ускорение)
FRAME_STEP = 2
# Размер, до которого уменьшаем кадр для модели (ускорение)
IMG_W, IMG_H = 1280, 720
# ROI — многоугольник (область интереса) в координатах исходного кадра
ROI_POLYGON = [
    (1100, 600),
    (1850, 650),
    (1650, 1000),
    (200, 1000)
]

# Площадь ROI в метрах квадратных (для плотности людей)
AREA_M2 = 150.0
# Фильтр по минимальной площади bbox (убираем мелкие/дальние шумные детекты)
MIN_BBOX_AREA = 900
# Окно сглаживания по количеству людей (moving average)
WINDOW = 15
# Устройство (для Apple Silicon обычно "mps")
DEVICE = "mps"
# half=True ускоряет на GPU (если поддерживается)
HALF = True
# Писать ли аннотированное видео (bbox, id, траектории, текст)
WRITE_ANNOTATED_VIDEO = True
# Максимальная длина истории траектории (в точках)
MAX_TRACK_HISTORY = 60
# Ограничение скорости (м/с), чтобы выбросы не ломали графики
SPEED_CLAMP_M_S = 8.0  # 8 м/с ≈ быстрый бег
# Длина хвоста траектории на видео (секунды)
TRAIL_SECONDS = 2.0
# Папка для результатов (по моделям будут подпапки)
OUT_DIR = Path("runs/crowd_eval")


# ------------------------------
# ROI utils
# ------------------------------

def make_roi_mask(h, w, polygon):
    """
    Создаём маску ROI (бинарная картинка), где ROI=255, остальное=0.
    Удобно, чтобы быстро проверять "точка внутри ROI или нет".
    """
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(polygon, dtype=np.int32)], 255)
    return mask

def box_center_in_roi(box, roi_mask):
    """
    Проверяем: центр bbox находится в ROI или нет.
    Возвращаем (inside, (cx, cy)).
    """
    x1, y1, x2, y2 = box
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)

    # на всякий случай клип по границам кадра
    cy = np.clip(cy, 0, roi_mask.shape[0] - 1)
    cx = np.clip(cx, 0, roi_mask.shape[1] - 1)

    return roi_mask[cy, cx] > 0, (cx, cy)

def polygon_area_px(polygon):
    """Площадь многоугольника в пикселях (по координатам кадра)."""
    pts = np.array(polygon, dtype=np.float32)
    return float(cv2.contourArea(pts))

def crowd_level(avg_count):
    """
    Очень простой классификатор "уровня загруженности"
    по среднему количеству людей в ROI.
    """
    if avg_count < 10:
        return "LOW"
    elif avg_count < 20:
        return "MEDIUM"
    else:
        return "HIGH"

def level_color(level):
    """
    Цвет для текста на видео под уровень загруженности.
    BGR (OpenCV).
    """
    if level == "LOW":
        return (0, 255, 0)
    elif level == "MEDIUM":
        return (0, 255, 255)
    else:
        return (0, 0, 255)


# ------------------------------
# Core processing
# ------------------------------

@dataclass
class RunResult:
    """
    Результаты одного прогона (одной модели):
    - per_frame: статистика по кадрам
    - tracks: точки треков (id + координаты + скорость)
    - summary: сводка метрик (proxy)
    - out_dir: папка результатов конкретной модели
    """
    per_frame: pd.DataFrame
    tracks: pd.DataFrame
    summary: dict
    out_dir: Path

def process_one_model(weights: str) -> RunResult:
    """
    Прогон одной модели по видео:
    - детект + трекинг людей (ByteTrack)
    - считаем людей в ROI
    - строим траектории и оценку скорости
    - сохраняем CSV, JSON и графики
    - (опционально) сохраняем аннотированное видео
    """
    out_dir = OUT_DIR / Path(weights).stem
    out_dir.mkdir(parents=True, exist_ok=True)

    cap = cv2.VideoCapture(VIDEO_PATH)
    assert cap.isOpened(), f"Видео не открылось: {VIDEO_PATH}"

    # FPS исходного видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps is None or fps <= 1e-6:
        fps = 30.0  # fallback, если метаданные битые

    # шаг времени между обработанными кадрами (с учётом FRAME_STEP)
    dt = FRAME_STEP / fps

    # читаем первый кадр, чтобы узнать размер
    ret, frame0 = cap.read()
    assert ret, "Первый кадр не читается"
    H, W = frame0.shape[:2]

    # маска ROI в оригинальном разрешении
    roi_mask = make_roi_mask(H, W, ROI_POLYGON)

    # Перевод пикселей в метры (очень грубо!)
    # Идея: если знаем площадь ROI в м² и площадь ROI в пикселях,
    # то длина "метров на пиксель" ≈ sqrt(AREA_M2 / ROI_area_px)
    roi_area_px = polygon_area_px(ROI_POLYGON)
    m_per_px = math.sqrt(AREA_M2 / max(roi_area_px, 1.0))

    # Писатель видео (если нужен)
    writer = None
    if WRITE_ANNOTATED_VIDEO:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(
            str(out_dir / "annotated.mp4"),
            fourcc,
            fps / FRAME_STEP,  # чтобы по времени проигрывание было примерно норм
            (W, H)
        )

    # Загружаем модель YOLO
    model = YOLO(weights)

    # Буфер для сглаживания count
    counts_buffer = deque(maxlen=WINDOW)

    # Сюда пишем строки per-frame статистики
    per_frame_rows = []

    # Сюда пишем точки треков (для траекторий/скорости/графиков)
    track_rows = []

    # История для траекторий: track_id -> deque[(t, x, y)]
    history = defaultdict(lambda: deque(maxlen=MAX_TRACK_HISTORY))

    # Главный цикл
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_idx = 0
    processed = 0

    # Таймер измерения скорости обработки (встроенный OpenCV)
    tickmeter = cv2.TickMeter()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        # Пропускаем кадры по шагу
        if frame_idx % FRAME_STEP != 0:
            continue

        processed += 1

        # Время кадра в секундах
        t_sec = frame_idx / fps
        t_min = int(t_sec // 60)

        tickmeter.start()

        # Уменьшаем кадр для модели (ускоряем инференс)
        frame_small = cv2.resize(frame, (IMG_W, IMG_H))

        # track() = детект + трекинг (ByteTrack)
        result = model.track(
            frame_small,
            conf=CONF_THRES,
            iou=IOU_THRES,
            imgsz=IMG_W,
            device=DEVICE,
            half=HALF,
            persist=True,               # важно для сохранения id между кадрами
            tracker="bytetrack.yaml",
            verbose=False
        )[0]

        tickmeter.stop()

        # FPS именно обработки (не видео)
        proc_time = tickmeter.getTimeSec()
        proc_fps = (1.0 / proc_time) if proc_time > 1e-9 else 0.0
        tickmeter.reset()

        # Счётчик людей в ROI на текущем кадре
        count = 0

        # Масштабирование координат bbox из frame_small обратно в оригинальный кадр
        scale_x = W / IMG_W
        scale_y = H / IMG_H

        # Рисуем ROI на кадре
        cv2.polylines(frame, [np.array(ROI_POLYGON, dtype=np.int32)], True, (255, 0, 0), 2)

        # Если есть bbox
        if result.boxes is not None and len(result.boxes) > 0:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            confs = result.boxes.conf.cpu().numpy() if result.boxes.conf is not None else np.zeros(len(boxes))

            # id треков (если трекер отдал)
            ids = None
            if result.boxes.id is not None:
                ids = result.boxes.id.cpu().numpy().astype(int)

            for i, (box, cls) in enumerate(zip(boxes, classes)):
                # cls==0 обычно person в COCO
                if cls != 0:
                    continue

                x1, y1, x2, y2 = box

                # bbox в координатах исходного кадра
                box_orig = [
                    int(x1 * scale_x),
                    int(y1 * scale_y),
                    int(x2 * scale_x),
                    int(y2 * scale_y),
                ]

                # фильтр мелких bbox
                bw = box_orig[2] - box_orig[0]
                bh = box_orig[3] - box_orig[1]
                area = bw * bh
                if area < MIN_BBOX_AREA:
                    continue

                # проверяем центр bbox на попадание в ROI
                inside, (cx, cy) = box_center_in_roi(box_orig, roi_mask)
                if not inside:
                    continue

                # это валидный человек в ROI
                count += 1

                track_id = int(ids[i]) if ids is not None else -1
                conf = float(confs[i])

                # --------------------------
                # Оценка скорости (м/с)
                # --------------------------
                # Считаем расстояние между текущей точкой и предыдущей точкой трека.
                # Это грубая оценка, потому что:
                # - m_per_px приближённый
                # - трекер может прыгать
                # - перспектива не учитывается
                speed_m_s = None
                if track_id != -1:
                    hist = history[track_id]
                    if len(hist) > 0:
                        prev_t, prev_x, prev_y = hist[-1]
                        px_dist = math.hypot(cx - prev_x, cy - prev_y)
                        meters = px_dist * m_per_px
                        s = meters / max(t_sec - prev_t, dt)

                        # clamp, чтобы выбросы не убивали графики
                        s = float(np.clip(s, 0.0, SPEED_CLAMP_M_S))
                        speed_m_s = s

                    # добавляем текущую точку в историю
                    hist.append((t_sec, cx, cy))

                # сохраняем точку трека
                track_rows.append({
                    "time_sec": t_sec,
                    "minute": t_min,
                    "track_id": track_id,
                    "cx": cx,
                    "cy": cy,
                    "conf": conf,
                    "speed_m_s": speed_m_s,
                })

                # Рисуем bbox + id + скорость
                cv2.rectangle(frame, (box_orig[0], box_orig[1]), (box_orig[2], box_orig[3]), (0, 255, 0), 2)
                label = f"id={track_id}" if track_id != -1 else "id=?"
                if speed_m_s is not None:
                    label += f" {speed_m_s:.1f} m/s"
                cv2.putText(
                    frame,
                    label,
                    (box_orig[0], max(20, box_orig[1] - 8)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 0),
                    2
                )

        # --------------------------
        # Сглаживание и плотность
        # --------------------------
        counts_buffer.append(count)
        avg_count = float(sum(counts_buffer) / len(counts_buffer))
        density = avg_count / AREA_M2
        level = crowd_level(avg_count)
        color = level_color(level)

        # --------------------------
        # Текст поверх видео
        # --------------------------
        # --- bbox label (где рисуешь id + скорость) ---

        # --- HUD поверх видео (где Model/People/Загруженность/Плотность/FPS) ---
        cv2.putText(frame, f"Model: {Path(weights).stem}", (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

        cv2.putText(frame, f"People: {count}", (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        cv2.putText(frame, f"Crowd level: {level} (avg={avg_count:.1f})", (30, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)

        cv2.putText(frame, f"Density: {density:.3f} ppl/m^2", (30, 160),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

        cv2.putText(frame, f"Processing FPS: {proc_fps:.1f}", (30, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)


        # --------------------------
        # Траектории (хвост за последние TRAIL_SECONDS)
        # --------------------------
        if WRITE_ANNOTATED_VIDEO:
            for tid, pts in history.items():
                if len(pts) < 2:
                    continue

                # удаляем точки, которые старше TRAIL_SECONDS
                while len(pts) > 0 and (t_sec - pts[0][0]) > TRAIL_SECONDS:
                    pts.popleft()

                if len(pts) < 2:
                    continue

                # рисуем хвост траектории
                poly = np.array([(x, y) for (_, x, y) in pts], dtype=np.int32)
                cv2.polylines(frame, [poly], False, (255, 255, 0), 2)

            writer.write(frame)

        # сохраняем строку per-frame статистики
        per_frame_rows.append({
            "frame_idx": frame_idx,
            "time_sec": t_sec,
            "minute": t_min,
            "count": count,
            "avg_count": avg_count,
            "density_ppl_m2": density,
            "proc_fps": proc_fps,
        })

    cap.release()
    if writer is not None:
        writer.release()

    per_frame = pd.DataFrame(per_frame_rows)
    tracks = pd.DataFrame(track_rows)

    # --------------------------
    # Сводные метрики (proxy)
    # --------------------------
    # Важно: без GT мы не можем посчитать mAP и т.д.
    # Поэтому тут "прокси": среднее число детекций, стабильность треков, FPS обработки и т.п.

    valid_tracks = tracks[tracks["track_id"] != -1].copy()
    track_len = valid_tracks.groupby("track_id").size() if len(valid_tracks) else pd.Series(dtype=int)

    summary = {
        "weights": weights,
        "video": VIDEO_PATH,
        "processed_frames": int(len(per_frame)),
        "duration_sec_est": float(per_frame["time_sec"].max()) if len(per_frame) else 0.0,
        "mean_count": float(per_frame["count"].mean()) if len(per_frame) else 0.0,
        "std_count": float(per_frame["count"].std(ddof=0)) if len(per_frame) else 0.0,
        "peak_count": int(per_frame["count"].max()) if len(per_frame) else 0,
        "mean_density_ppl_m2": float(per_frame["density_ppl_m2"].mean()) if len(per_frame) else 0.0,
        "mean_proc_fps": float(per_frame["proc_fps"].replace([np.inf, -np.inf], np.nan).dropna().mean()) if len(per_frame) else 0.0,
        "unique_track_ids": int(valid_tracks["track_id"].nunique()) if len(valid_tracks) else 0,
        "mean_track_len_obs": float(track_len.mean()) if len(track_len) else 0.0,
        "median_track_len_obs": float(track_len.median()) if len(track_len) else 0.0,
        "mean_speed_m_s": float(valid_tracks["speed_m_s"].dropna().mean()) if "speed_m_s" in valid_tracks and len(valid_tracks) else 0.0,
        "p95_speed_m_s": float(valid_tracks["speed_m_s"].dropna().quantile(0.95)) if "speed_m_s" in valid_tracks and len(valid_tracks["speed_m_s"].dropna()) else 0.0,
        "m_per_px_used": float(m_per_px),
    }

    # сохраняем сырые результаты
    per_frame.to_csv(out_dir / "per_frame.csv", index=False)
    tracks.to_csv(out_dir / "tracks.csv", index=False)
    with open(out_dir / "summary.json", "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    # графики
    plot_counts_per_5sec(per_frame, out_dir, Path(weights).stem)

    return RunResult(per_frame=per_frame, tracks=tracks, summary=summary, out_dir=out_dir)


# ------------------------------
# Plotting
# ------------------------------

def plot_counts_per_5sec(per_frame: pd.DataFrame, out_dir: Path, model: str):
    """
    График количества людей с бинами по 5 секунд.
    По X — время, красиво в формате мм:сс.
    """
    if len(per_frame) == 0:
        return

    df = per_frame.copy()

    # Бин по 5 секунд (0,5,10,15...)
    df["t5"] = (df["time_sec"] // 5).astype(int) * 5

    # Можно mean/max — тут оставим оба + сглаженное
    agg = df.groupby("t5").agg(
        mean_count=("count", "mean"),
        mean_smooth=("avg_count", "mean") if "avg_count" in df.columns else ("count", "mean"),
        max_count=("count", "max")
    ).reset_index()

    def fmt_mmss(x, pos=None):
        x = int(round(x))
        m = x // 60
        s = x % 60
        return f"{m:02d}:{s:02d}"

    plt.figure()
    plt.plot(agg["t5"], agg["mean_count"], label="среднее за 5 сек")
    plt.plot(agg["t5"], agg["max_count"], label="максимум за 5 сек")
    plt.plot(agg["t5"], agg["mean_smooth"], label="сглаженное")

    ax = plt.gca()
    ax.xaxis.set_major_locator(MultipleLocator(5))        # деление = 5 секунд
    ax.xaxis.set_major_formatter(FuncFormatter(fmt_mmss)) # формат 00:05

    ax.set_xlabel("Время (мм:сс)")
    ax.set_ylabel("Люди в ROI")
    ax.set_title(f"Количество людей во времени (шаг = 5 сек), model = {model}")

    # Если хочешь, можно убрать лимиты. Сейчас оставил как было.
    ax.set_ylim(10, 30)

    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "counts_per_5sec.png", dpi=160)
    plt.close()

def plot_model_comparison(summaries: list[dict], out_dir: Path):
    """
    Сравнение моделей (прокси):
    - X: средний FPS обработки
    - Y: среднее количество людей в ROI (как прокси “находит больше/меньше”)
    """
    df = pd.DataFrame(summaries)
    df.to_csv(out_dir / "model_comparison.csv", index=False)

    plt.figure()
    plt.scatter(df["mean_proc_fps"], df["mean_count"])
    for _, r in df.iterrows():
        plt.text(r["mean_proc_fps"], r["mean_count"], Path(r["weights"]).stem)

    plt.xlabel("Средний FPS обработки")
    plt.ylabel("Среднее число людей в ROI")
    plt.title("Сравнение моделей: скорость vs количество детекций (прокси)")
    plt.tight_layout()
    plt.savefig(out_dir / "model_comparison.png", dpi=160)
    plt.close()


# ------------------------------
# Main
# ------------------------------

def main():
    # Создаём папку под результаты
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    summaries = []
    for w in MODEL_WEIGHTS_LIST:
        print(f"\n=== Запуск {w} ===")
        res = process_one_model(w)
        summaries.append(res.summary)
        print(f"Сохранено в: {res.out_dir}")

    plot_model_comparison(summaries, OUT_DIR)
    print("\nГотово. Смотри папку runs/crowd_eval/")

if __name__ == "__main__":
    main()



=== Запуск yolo11n.pt ===
Сохранено в: runs/crowd_eval/yolo11n

=== Запуск yolo11s.pt ===
Сохранено в: runs/crowd_eval/yolo11s

=== Запуск yolo11m.pt ===
Сохранено в: runs/crowd_eval/yolo11m

=== Запуск yolo11l.pt ===
Сохранено в: runs/crowd_eval/yolo11l

=== Запуск yolo11x.pt ===
Сохранено в: runs/crowd_eval/yolo11x

Готово. Смотри папку runs/crowd_eval/


In [19]:
from pathlib import Path

def _df_from_summaries(summaries: list[dict]) -> pd.DataFrame:
    df = pd.DataFrame(summaries).copy()
    df["model"] = df["weights"].apply(lambda w: Path(w).stem)
    # на всякий пожарный
    for c in df.columns:
        if c != "model" and c != "weights" and c != "video":
            df[c] = pd.to_numeric(df[c], errors="ignore")
    return df

def plot_model_bars(summaries: list[dict], out_dir: Path):
    """
    Набор барчартов по summary.json:
    - FPS (скорость)
    - mean_count / peak_count (нагрузка)
    - std_count (шумность)
    - median_track_len_obs (стабильность трекинга прокси)
    - p95_speed_m_s (выбросы скорости = дрожание трека)
    """
    df = _df_from_summaries(summaries)
    df = df.sort_values("mean_proc_fps", ascending=False)

    def bar(metric: str, title: str, ylabel: str, filename: str):
        if metric not in df.columns:
            return
        plt.figure()
        plt.bar(df["model"], df[metric])
        plt.xlabel("Модель")
        plt.ylabel(ylabel)
        plt.title(title)
        plt.tight_layout()
        plt.savefig(out_dir / filename, dpi=160)
        plt.close()

    bar("mean_proc_fps", "Скорость обработки (средний FPS)", "FPS", "bar_mean_proc_fps.png")
    bar("mean_count", "Среднее число людей в ROI (прокси)", "людей", "bar_mean_count.png")
    bar("peak_count", "Пиковое число людей в ROI", "людей", "bar_peak_count.png")
    bar("std_count", "Шумность детекции: std(count) (меньше = стабильнее)", "std", "bar_std_count.png")
    bar("median_track_len_obs", "Стабильность трекинга (прокси): медиана длины трека", "наблюдений", "bar_median_track_len.png")
    bar("p95_speed_m_s", "Выбросы скорости (p95): чем выше — тем больше прыжков/дрожания", "м/с", "bar_p95_speed.png")


def plot_tradeoff_bubbles(summaries: list[dict], out_dir: Path):
    """
    Bubble chart:
    X = mean_proc_fps
    Y = mean_count
    Размер пузыря = median_track_len_obs (стабильность трека прокси)
    """
    df = _df_from_summaries(summaries)

    x = df["mean_proc_fps"].values
    y = df["mean_count"].values

    # bubble size
    if "median_track_len_obs" in df.columns:
        s_raw = df["median_track_len_obs"].fillna(0).values
    else:
        s_raw = np.ones(len(df))

    # нормируем размеры, чтобы было красиво
    s = 200 + 1200 * (s_raw - s_raw.min()) / (max(1e-9, (s_raw.max() - s_raw.min())))

    plt.figure()
    plt.scatter(x, y, s=s, alpha=0.7)
    for _, r in df.iterrows():
        plt.text(r["mean_proc_fps"], r["mean_count"], r["model"])
    plt.xlabel("Средний FPS обработки")
    plt.ylabel("Среднее число людей в ROI (прокси)")
    plt.title("Trade-off: скорость vs детекции (пузырь = стабильность трека)")
    plt.tight_layout()
    plt.savefig(out_dir / "tradeoff_bubbles.png", dpi=160)
    plt.close()


def plot_metrics_heatmap(summaries: list[dict], out_dir: Path):
    """
    Heatmap по ключевым метрикам.
    Делает min-max нормализацию, чтобы всё было на одной шкале.
    """
    df = _df_from_summaries(summaries)

    # выбери метрики, которые у тебя точно есть в summary.json
    metrics = [
        "mean_proc_fps",
        "mean_count",
        "std_count",
        "median_track_len_obs",
        "p95_speed_m_s",
        "mean_density_ppl_m2",
    ]
    metrics = [m for m in metrics if m in df.columns]
    if not metrics:
        return

    M = df[metrics].astype(float).values

    # min-max по столбцам
    mn = np.nanmin(M, axis=0)
    mx = np.nanmax(M, axis=0)
    denom = np.where((mx - mn) < 1e-9, 1.0, (mx - mn))
    Z = (M - mn) / denom

    plt.figure()
    plt.imshow(Z, aspect="auto")
    plt.yticks(range(len(df)), df["model"].tolist())
    plt.xticks(range(len(metrics)), metrics, rotation=30, ha="right")
    plt.title("Сравнение моделей по метрикам (min-max нормализация)")
    plt.colorbar(label="0..1")
    plt.tight_layout()
    plt.savefig(out_dir / "metrics_heatmap.png", dpi=160)
    plt.close()


def plot_pareto_front(summaries: list[dict], out_dir: Path):
    """
    Pareto (наглядно для презы):
    "лучше" = больше FPS и больше mean_count.
    Отмечаем точки, которые не доминируются другими.
    """
    df = _df_from_summaries(summaries).copy()
    df = df.dropna(subset=["mean_proc_fps", "mean_count"])

    pts = df[["mean_proc_fps", "mean_count"]].values
    is_pareto = np.ones(len(df), dtype=bool)

    for i in range(len(df)):
        for j in range(len(df)):
            if i == j:
                continue
            # j доминирует i, если >= по обоим и > хотя бы по одному
            if (pts[j][0] >= pts[i][0] and pts[j][1] >= pts[i][1]) and (pts[j][0] > pts[i][0] or pts[j][1] > pts[i][1]):
                is_pareto[i] = False
                break

    plt.figure()
    plt.scatter(df["mean_proc_fps"], df["mean_count"])
    for _, r in df.iterrows():
        plt.text(r["mean_proc_fps"], r["mean_count"], r["model"])

    # выделим pareto точки
    pareto_df = df[is_pareto]
    plt.scatter(pareto_df["mean_proc_fps"], pareto_df["mean_count"], s=200, alpha=0.8)

    plt.xlabel("Средний FPS обработки")
    plt.ylabel("Среднее число людей в ROI (прокси)")
    plt.title("Pareto-фронт: лучшие компромиссы скорость/детекции")
    plt.tight_layout()
    plt.savefig(out_dir / "pareto_front.png", dpi=160)
    plt.close()


In [23]:
import json
from pathlib import Path

OUT_DIR = Path("runs/crowd_eval")

summaries = []
for p in sorted(OUT_DIR.glob("*/summary.json")):
    with open(p, "r", encoding="utf-8") as f:
        summaries.append(json.load(f))

print("Загружено моделей:", len(summaries))
[s["weights"] for s in summaries]

plot_model_bars(summaries, OUT_DIR)
plot_tradeoff_bubbles(summaries, OUT_DIR)
plot_metrics_heatmap(summaries, OUT_DIR)
plot_pareto_front(summaries, OUT_DIR)

print("Готово! PNG лежат в:", OUT_DIR)

Загружено моделей: 5


/var/folders/g6/6x5w0yy121z4xzhc9ncs905m0000gn/T/ipykernel_8131/3368360561.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


Готово! PNG лежат в: runs/crowd_eval


/var/folders/g6/6x5w0yy121z4xzhc9ncs905m0000gn/T/ipykernel_8131/3368360561.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
/var/folders/g6/6x5w0yy121z4xzhc9ncs905m0000gn/T/ipykernel_8131/3368360561.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
/var/folders/g6/6x5w0yy121z4xzhc9ncs905m0000gn/T/ipykernel_8131/3368360561.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [ ]:
3) Про стабильность трекинга (это твой козырь)
Смотри на median_track_len_obs (медиана длины трека):
n: 19
s: 23
m: 23.5
l: 29.5 ← лучший по стабильности
x: 25
То есть yolo11l реально даёт более “длинные” треки (меньше рвёт траектории), но ценник — 16 FPS.
А unique_track_ids уменьшается от n → l:
n: 463, s: 408, m: 394, l: 372
Это можно трактовать так: “на больших моделях трекер меньше дробит людей на новые ID” (обычно это хорошо).